Author:      Abhijeet Singh

Copyright:   Copyright (©) 2018 EPFL (Ecole Polytechnique Fédérale de Lausanne)
             Geo-Energy lab

# Circular Hole in an Infinite Medium Subjected to Uniaxial Tension

Set the Bigwham directory (check on yours)

In [1]:
import os, sys
home = os.environ["HOME"]
sys.path.append(home + "/BigWham/build/interfaces/python")

Import necessary libraries

In [2]:
import numpy as np
from hmatrix_rectangular import HmatrixRectangular
import matplotlib.pyplot as plt
from scipy.sparse.linalg import gmres, LinearOperator
from scipy.sparse import csr_matrix
from py_bigwham import Mesh

# User Inputs

In [3]:
# Solving the Kirsch's problem

# Circle radius
r = 1          
# Number of elements in the circle 
n_elmts = 40
# Angle spanned by each element
theta = 2*np.pi/n_elmts

# Create the mesh of circle
coord = np.zeros([n_elmts,2])
for i in range(n_elmts):
    # Assign x-coordinates (intentionally shifting by theta/2, which makes easy to put BCs on colocation points)
    coord[i,0] = r*np.cos(theta*i - theta/2)
    # Assign y-coordinates (intentionally shifting by theta/2, which makes easy to put BCs on colocation points)
    coord[i,1] = r*np.sin(theta*i - theta/2)

# Make the connectivity matrix
conn = np.fromfunction(lambda x, y: x + y, (n_elmts, 2), dtype=np.int_)
# Put the last element with 0th node as the end node because it's a circle
conn[-1,1] = conn[0,0]


H-matrix parameters

In [4]:
# H-matrix parameter

max_leaf_size = 16
eta           = 3.0
eps_aca       = 1.e-4

Elastic properties

In [5]:
# Elastic properties

# Young's Modulus
E     = 1
# Poisson's Ratio
nu    = 0.2

Compute H, T, and U matrices

In [6]:
# Compute H, T and U matrices 

H = HmatrixRectangular(
    "2DS0-2DS0-H",
    coord.flatten(),
    conn.flatten(),
    coord.flatten(),
    conn.flatten(),
    np.array([E, nu]),
    max_leaf_size,
    eta,
    eps_aca,
)

T = HmatrixRectangular(
    "2DS0-2DS0-T",
    coord.flatten(),
    conn.flatten(),
    coord.flatten(),
    conn.flatten(),
    np.array([E, nu]),
    max_leaf_size,
    eta,
    eps_aca,
)


U = HmatrixRectangular(
    "2DS0-2DS0-U",
    coord.flatten(),
    conn.flatten(),
    coord.flatten(),
    conn.flatten(),
    np.array([E, nu]),
    max_leaf_size,
    eta,
    eps_aca,
)


 Now setting things for kernel ... 2DS0-2DS0-H with properties size 2
Cluster tree creation time for the source mesh :  0
Cluster tree creation time for the source mesh :  0
Time for binary cluster tree construction  2.1364e-05
 binary cluster tree depth =2
 Number of blocks =16
 Number of full blocks =16
 Number of low rank blocks =0
 Loop on full blocks construction  
 N full blocks 16 
Loop on low rank blocks construction
N low rank blocks 0
dof_dimension: 2
Creation of hmat done in 0.00206868
Compression ratio - 1
Hmat object - built 
HMAT --> built 
HMAT set, CR = 1, eps_aca = 0.0001, eta = 3
BigWhamIO ENDED
 Now setting things for kernel ... 2DS0-2DS0-T with properties size 2
Cluster tree creation time for the source mesh :  0
Cluster tree creation time for the source mesh :  0
Time for binary cluster tree construction  1.9301e-05
 binary cluster tree depth =2
 Number of blocks =16
 Number of full blocks =16
 Number of low rank blocks =0
 Loop on full blocks construction  
 N ful

Get traction on crack surface using overall stress field and initialize displacement field

In [7]:
# Get the mesh from coordinates and connectivity
mesh = Mesh(coord.flatten(), conn.flatten(), "2DS0")
# Get the collocation points
col_pts = H.getMeshCollocationPoints()

# Individual stresses
sigma_11 = 0.0
sigma_22 = 1.0
sigma_12 = 0.0
# Overall stress field
sigma = np.array([sigma_11, sigma_22, sigma_12], dtype=np.float64)

# Get traction using an internal function
t = -mesh.get_farfield_traction(sigma)
# Initialize the displacement field
u = np.zeros(n_elmts*2)

Choose indices with pre-specified displacement and traction BCs, respectively 

In [13]:
# Declare all indices to later slice accordingly
all_indices  = np.arange(n_elmts*2)
# The indices where displacement BC is given
# Point1: [r, 0]
# Point2: [-r, 0]
disp_indices = np.array([0, 1, n_elmts, n_elmts+1], dtype=int)
# The indices where traction BC is given
trac_indices = np.setdiff1d(all_indices, disp_indices)

Create vector masks where displacement and traction BCs, respectively, are specified

In [14]:
# Create mask for indices where displacement BC is given
mask_disp = np.zeros(n_elmts*2, dtype=bool)
mask_disp[disp_indices] = True

# Create mask for indices where traction BC is given
mask_trac = np.zeros(n_elmts*2, dtype=bool)
mask_trac[trac_indices] = True

Matvec operation and preconditioner to solve the system of equations

$$
\begin{Bmatrix}
        t_1\\t_2^*
        \end{Bmatrix}\,=\,
        \begin{bmatrix}
        T_{11} & T_{12} \\
        T_{21} & T_{22}
        \end{bmatrix}
        \begin{Bmatrix}
        t_1 \\ t_2^*
        \end{Bmatrix} - 
        \begin{bmatrix}
        H_{11} & H_{12} \\
        H_{21} & H_{22}
        \end{bmatrix}
        \begin{Bmatrix}
        u_1^* \\ u_2
        \end{Bmatrix}
$$

which then becomes,
$$
\begin{bmatrix}
        H_{11} & 0 \\
        H_{21} & 0
        \end{bmatrix}
        \begin{Bmatrix}
        u_1^* \\ 0
        \end{Bmatrix} -
        \begin{bmatrix}
        0 & T_{12} \\
        0 & T_{22}
        \end{bmatrix}
        \begin{Bmatrix}
        0 \\ t_2^*
        \end{Bmatrix} +
        \begin{Bmatrix}
        0 \\ t_2^*
        \end{Bmatrix} = 
        - \begin{bmatrix}
        0 & H_{12} \\
        0 & H_{22}
        \end{bmatrix}
        \begin{Bmatrix}
        0 \\ u_2
        \end{Bmatrix} +
        \begin{bmatrix}
        T_{11} & 0 \\
        T_{21} & 0
        \end{bmatrix}
        \begin{Bmatrix}
        t_1 \\ 0
        \end{Bmatrix} - 
        \begin{Bmatrix}
        t_1 \\ 0
        \end{Bmatrix}
$$

LHS (unknown) part of the equation is given as:
$$
\begin{bmatrix}
        H_{11} & H_{12} \\
        H_{21} & H_{22}
        \end{bmatrix}
        \begin{bmatrix}
        I_{11} & 0 \\
        0 & 0
        \end{bmatrix}
        \begin{Bmatrix}
        u_1^* \\ t_2^*
        \end{Bmatrix} - 
        \begin{bmatrix}
        T_{11} & T_{12} \\
        T_{12} & T_{22}
        \end{bmatrix}
        \begin{bmatrix}
        0 & 0 \\
        0 & I_{22}
        \end{bmatrix}
        \begin{Bmatrix}
        u_1^* \\ t_2^*
        \end{Bmatrix} ~~ + ~~ 
        \begin{bmatrix}
        0 & 0 \\
        0 & I_{22}
        \end{bmatrix}
        \begin{Bmatrix}
        u_1^* \\ t_2^*
        \end{Bmatrix}
$$

where $u_1^*$ and $t_2^*$ are unknown displacements and unknown tractions, respectively.

In [15]:
# Index 1 is where traction is known
# Index 2 is where displacement is known 

# Define the system to solve for unknown tractions and displacements

# Equation: H11xu1 - T12      xt2 = -H12xu2 + (T11-I11)xt1
#           H21xu1 - (T22-I22)xt2 = -H22xu2 +  T21     xt1
def matvec(x: np.ndarray) -> np.ndarray:
    # Get [0, t2]^T
    x_trac = x*mask_disp
    # Get [u1, 0]^T
    x_disp = x*mask_trac
    
    y1 = H @ x_disp
    y2 = -T @ x_trac + x_trac
    return y1+y2

# To obtain LHS part of equation
soln = LinearOperator(shape=T.shape, matvec=matvec)
# Compute the inverse of the diagonal as a Jacobi preconditioner
diagonal                = np.diag(matvec(np.eye(np.shape(T)[0])))
inverse_diagonal        = 1.0 / diagonal
jacobi_preconditioner   = LinearOperator(np.shape(T), matvec=lambda v: inverse_diagonal * v)


Solve the system of equations

RHS (known) part of the equation is given as:
$$
- \begin{bmatrix}
        H_{11} & H_{12} \\
        H_{21} & H_{22}
        \end{bmatrix}
        \begin{bmatrix}
        0 & 0 \\
        0 & I_{22}
        \end{bmatrix}
        \begin{Bmatrix}
        t_1 \\ u_2
        \end{Bmatrix} + 
        \begin{bmatrix}
        T_{11} & T_{12} \\
        T_{12} & T_{22}
        \end{bmatrix}
        \begin{bmatrix}
        I_{11} & 0 \\
        0 & 0
        \end{bmatrix}
        \begin{Bmatrix}
        t_1 \\ u_2
        \end{Bmatrix} ~~ - ~~ 
        \begin{bmatrix}
        I_{11} & 0 \\
        0 & 0
        \end{bmatrix}
        \begin{Bmatrix}
        t_1 \\ u_2
        \end{Bmatrix}
$$

where $t_1$ and $u_2$ are known tractions and known displacements, respectively.

In [16]:
# Index 1 is where traction is known
# Index 2 is where displacement is known 


# Get local traction from global traction
t_local  = mesh.convert_to_local(t)
# Tranform using mask to get non-zero values only at index 1
t1_local = t_local*mask_trac
# Get local displacement from global displacement
u_local  = mesh.convert_to_local(u)
# Tranform using mask to get non-zero values only at index 2
u2_local = u_local*mask_disp


# Part 1:
# b_local1 is -[H12xu2, H22xu2]^T
b_local1 = -H @ u2_local

# b_local2 is [T11xt1 - t1, T21xt1]^T
b_local2 = T @ t1_local - t1_local


# Combine all to get b_local
b_local = b_local1 + b_local2

# Get [u1_local, t2_local]^T by inverting the equation
# H11xu1 -  T12     xt2 = -H12xu2 + (T11-I11)xt1
# H21xu1 - (T22-I22)xt2 = -H22xu2 +  T21     xt1
disp_trac = gmres(soln, b_local, M = jacobi_preconditioner)[0]
# Extract the unknown diplacements and tractions
u1_local = disp_trac*mask_trac
t2_local = disp_trac*mask_disp
# Get the final global displaments and tractions
u_global = mesh.convert_to_global(u1_local+u2_local)
t_global = mesh.convert_to_global(t1_local+t2_local)
